# 实现LeNet

In [1]:
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

## LeNet结构
![LeNet网络结构](./pictures/LeNet_structure.png)

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, stride = 2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
    
    def forward(self, X):
        temp = self.conv(X)
        return self.fc(temp.view(X.shape[0], -1))

In [3]:
test_net = LeNet()
print(test_net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


## 使用Fashion-MNIST测试网络

In [4]:
mnist_train = torchvision.datasets.FashionMNIST('./data/FashionMNIST', train= True, transform= transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST('./data/FashionMNIST', train= False, transform= transforms.ToTensor())

train_data = torch.utils.data.DataLoader(mnist_train, batch_size= 256, shuffle= True, num_workers= 4)
test_data = torch.utils.data.DataLoader(mnist_test, batch_size= 256, shuffle= True, num_workers= 4)

## 定义评估函数

In [5]:
def evaluate(data_iter, net):
    acc_sum, n = 0.0, 0
    device = list(net.parameters())[0].device
    net.eval()
    for X, y in data_iter:
        acc_sum += (net(X.to(device)).argmax(dim = 1) == y.to(device)).float().sum().cpu().item()
        n += y.shape[0]
    net.train()
    return acc_sum / n

## 损失函数使用交叉熵，优化算法为Adma

In [7]:
lr = 0.003
num_epochs = 10


net = LeNet()
net.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr)

device = ('cuda' if torch.cuda.is_available() else 'cpu')


for epoch in range(1, num_epochs + 1):
    for X, y in train_data:
        X = X.to(device)
        y = y.to(device)
        l = loss(net(X), y)
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    
    print("epoch %d train accuracy %f test accuracy %f" % (epoch, evaluate(train_data, net), evaluate(test_data, net)))

epoch 1 train accuracy 0.683633 test accuracy 0.678800
epoch 2 train accuracy 0.742933 test accuracy 0.739200
epoch 3 train accuracy 0.779633 test accuracy 0.772400
epoch 4 train accuracy 0.799383 test accuracy 0.785800
epoch 5 train accuracy 0.802183 test accuracy 0.790800
epoch 6 train accuracy 0.825433 test accuracy 0.811500
epoch 7 train accuracy 0.844567 test accuracy 0.833500
epoch 8 train accuracy 0.845083 test accuracy 0.835400
epoch 9 train accuracy 0.855667 test accuracy 0.845000
epoch 10 train accuracy 0.858367 test accuracy 0.847100
